In [14]:
%load_ext autoreload
%autoreload 2

import deltas
from deltas.pipeline import data, classifier, evaluation
from deltas.model import downsample
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
# np.random.seed(10)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

costs = (1, 1)  # change for (1, 10) to increase results

datasets = {0: 'Breast Cancer', 2: 'Iris', 3: 'Wine', 4: 'Pima Indian Diabetes',
            5: 'Sonar Rocks vs Mines', 6: 'Banknote Authentication',
            7: 'Abalone Gender', 8: 'Ionosphere', 9: 'Wheat Seeds',
            10: 'Credit Scoring 1', 11: 'Credit Scoring 2',
            12: 'Direct Marketing', 13: 'Habermans breast cancer',
            14: 'Wisconsin Breast Cancer', 15: 'Hepatitis',
            16: 'Heart Disease'}

dataset = datasets[4]  # change ind to select dataset to use
model = 'SVM-rbf'
# model = 'MLP'
# model = 'Linear'

# dataset = datasets[7]  # change ind to select dataset to use
dfs = []
data_clf_old = data.get_real_dataset(dataset, _print=False, scale=True)
for i in tqdm(range(10)):
    # np.random.seed(random.randint)
    # np.random.seed(i)
    data_clf = data.get_real_dataset(dataset, _print=False, seed=i, scale=True)
    # data_clf = data.get_real_dataset(dataset, _print=False, scale=True)
    print((data_clf['data']['X'] == data_clf_old['data']['X']).all())
    data_clf_old = data_clf
    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False,
        _print=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    if False:
        param_grid = {
                    #   'alpha': [0, 0.1, 1, 10],
                    #   'grid_search': [True, False],
                    'method': ['supports-prop-update_mean', 'supports-prop-update_mean-margin_only']}
        grid_original = GridSearchCV(
            downsample.downsample_deltas(), param_grid, refit=True)
        grid_original.fit(X, y,
                        clf=clf,
                        _print=False,
                        _plot=False,
                        max_trials=10000,
                        parallel=True)
        deltas_model = grid_original.best_estimator_
        print(f'Best params: {grid_original.best_params_}')
    else:
        deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                             alpha=10,
                                                             _print=False,
                                                             _plot=False,
                                                             method='supports-prop-update_mean',
                                                             max_trials=10000,
                                                             parallel=True)

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=False, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')

  0%|          | 0/10 [00:00<?, ?it/s]

False


 10%|█         | 1/10 [00:01<00:17,  1.93s/it]

not fit deltas
False


 20%|██        | 2/10 [00:05<00:23,  2.96s/it]

...

 80%|████████  | 8/10 [00:18<00:04,  2.04s/it]

not fit deltas
False


 90%|█████████ | 9/10 [00:21<00:02,  2.34s/it]

False


100%|██████████| 10/10 [00:23<00:00,  2.38s/it]


In [16]:
print(len(dfs))

5


In [17]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)
print(dataset)

m = mean_df.to_dict('list')
s = std_df.to_dict('list')
metrics = mean_df.columns.to_list()
methods = mean_df.index.to_list()
sf = 5
for metric in metrics:
    means = m[metric]
    sts = s[metric]
    mx = np.argmax(means)
    for i in range(len(means)):
        m_str = str(means[i])[1:sf]
        if i == mx:
            m_str = f"\\textbf{{{m_str}}}"
        s_str = str(sts[i])[1:sf-1]
        m[metric][i] = f'${m_str} \\pm {s_str}$'

method_map = {
    'Baseline': 'Baseline',
    'SMOTE': "SMOTE \cite{Chawla_2002_JAIR}",
    'Balanced Weights': 'BW',
    'BMR': 'BMR \cite{Bahnsen_2014_SIAM}',
    'Threshold': 'Thresh \cite{Sheng_2006_AAAI}',
    'Our Method': 'Our Method',
}
meths_new = []
for me in methods:
    meths_new.append(method_map[me])
m['Methods'] = meths_new
df = pd.DataFrame(m)  # .set_index('Methods')
meths = df.pop('Methods')
df.insert(0, 'Methods', meths)
latex_str = df.to_latex(index=False)

print('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])

Pima Indian Diabetes
\begin{tabular}{@{}lrrrr@{}}\toprule
Methods & Accuracy & G-Mean & F1 \\
\midrule
Baseline & $.673 \pm .02$ & $.458 \pm .18$ & $.352 \pm .19$ \\
SMOTE \cite{Chawla_2002_JAIR} & $.674 \pm .02$ & $.577 \pm .06$ & $.480 \pm .08$ \\
BW & $.673 \pm .02$ & $.458 \pm .18$ & $.352 \pm .19$ \\
BMR \cite{Bahnsen_2014_SIAM} & $\textbf{.704} \pm .03$ & $.674 \pm .04$ & $.604 \pm .05$ \\
Thresh \cite{Sheng_2006_AAAI} & $.699 \pm .04$ & $\textbf{.675} \pm .04$ & $\textbf{.614} \pm .04$ \\
Our Method & $.703 \pm .03$ & $.659 \pm .05$ & $.586 \pm .06$ \\
\bottomrule
\end{tabular}

